In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import math
import time
import os

In [2]:
tf.__version__

'1.2.1'

In [3]:
BATCH_SIZE = 50
num_units = 512
MAX_SEQLEN = 10
encoder_vocabulary_size = 23
encoder_embedding_size = 10
decoder_vocabulary_size = 23
decoder_embedding_size = 10
NLAYERS = 3


In [4]:
cells = [tf.contrib.rnn.GRUCell(num_units) for _ in range(NLAYERS)]
encoder_cell = tf.contrib.rnn.MultiRNNCell(cells)
ENCODER_INPUTS = tf.placeholder(dtype=tf.int32,shape=[None,None],name="Encoder_inputs") #BATCH_SIZE X SEQLEN
encoder_embedings = tf.get_variable(name="ENCODER_EMBEDDINGS",shape=[encoder_vocabulary_size,encoder_vocabulary_size],initializer=tf.contrib.layers.xavier_initializer(uniform=True,seed=None,dtype=tf.float32),
                                     dtype=tf.float32)
embed = tf.nn.embedding_lookup(encoder_embedings,ENCODER_INPUTS)

In [5]:
EncoderSeqLength= tf.placeholder(dtype=tf.int32,shape=[None])
output,state = tf.nn.dynamic_rnn(encoder_cell,embed,dtype=tf.float32,sequence_length=EncoderSeqLength)


In [9]:
d_state

AttentionWrapperState(cell_state=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 512) dtype=float32>, attention=<tf.Tensor 'AttentionWrapperZeroState/zeros_1:0' shape=(50, 512) dtype=float32>, time=<tf.Tensor 'AttentionWrapperZeroState/zeros:0' shape=() dtype=int32>, alignments=<tf.Tensor 'AttentionWrapperZeroState/zeros_2:0' shape=(50, ?) dtype=float32>, alignment_history=())

In [7]:

DecoderSeqLength = tf.placeholder(dtype=tf.int32,shape=[None]);
DECODER_INPUTS = tf.placeholder(dtype=tf.int32,shape=[None,None],name="DecoderInputs")

decoder_embeddings = tf.get_variable(name="DECODER_EMBEDDINGS",shape=[decoder_vocabulary_size,decoder_embedding_size],initializer=tf.contrib.layers.xavier_initializer(uniform=True,seed=None,dtype=tf.float32),
                                     dtype=tf.float32)
decoder_embed = tf.nn.embedding_lookup(decoder_embeddings,DECODER_INPUTS)
helper = tf.contrib.seq2seq.TrainingHelper(decoder_embed,sequence_length=DecoderSeqLength)
decoder_output_layer = tf.contrib.keras.layers.Dense(decoder_vocabulary_size)



In [8]:
####Apply Attention
##creates a class providing functionality of storing the memory(Encoder output states) and the depth of memory to which the query will be done .
attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(num_units,output)
#multi layer decoder cell
d_cells = [tf.contrib.rnn.GRUCell(num_units) for _ in range(NLAYERS)]
# apply attention on the top/last layer of the decoder cell and provide it as a wrapper . Output attention true then return attention else if false, then output = output
#Working of attention mechanism :
attention_cell = tf.contrib.seq2seq.AttentionWrapper(d_cells[-1],attention_mechanism,output_attention=False)
d_cells[-1] = attention_cell
decoder_cell = tf.contrib.rnn.MultiRNNCell(d_cells)
## shape of decoder cell --  (NLayer-1) * RNNCell + AttentionWrappedCell
## since the last layer in the cell is of type wrapped, so the last layer in the encoder final state should also be of form wrapped . So modify last layer state to be of type wrapped
d_state = attention_cell.zero_state(BATCH_SIZE,dtype=tf.float32)
decoder_input_state = [x for x in state] # convert tuple to array so that item assignment can be done later
d_state = d_state.clone(cell_state=decoder_input_state[-1])
decoder_input_state[-1] = d_state
decoder_input_state = tuple(decoder_input_state)
basic_decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell,helper,decoder_input_state,decoder_output_layer)
final_outputs, final_state, final_sequence_lengths= tf.contrib.seq2seq.dynamic_decode(basic_decoder)

In [ ]:
##decoder_logits = tf.contrib.layers.linear(final_outputs.rnn_output,decoder_vocabulary_size)
decoder_logits = final_outputs.rnn_output

In [ ]:
decoder_logits

In [ ]:
decoder_prediction = tf.arg_max(tf.nn.softmax(decoder_logits),2)

In [ ]:
decoder_targets = tf.placeholder(dtype=tf.int32,shape=[None,None],name = "DecoderTargets")
lr = tf.placeholder(dtype=tf.float32,name='lr')

In [ ]:
stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(decoder_targets,depth=decoder_vocabulary_size,dtype=tf.float32),logits=decoder_logits)




In [ ]:
loss = tf.reduce_mean(stepwise_cross_entropy)

In [ ]:
optimiser = tf.train.AdamOptimizer(lr).minimize(loss)

In [ ]:
source = open("sources.txt")
train = [[int(word) for word in line.split()] for line in source]
#train = np.array(train)

In [ ]:
target = open("targets.txt")
target = [[int(word) for word in line.split()] for line in target]
#targetValue = np.array(targetValue)

In [ ]:
#Process train data
EOS = 20
PAD = 21
GO = 22
epoch = 20
def modifyBatch(input):
    inputSeqLen = [len(seq) for seq in input]
    maxSourceSeqLen = max(inputSeqLen)
    batchedInput = np.ones(shape=(len(input),maxSourceSeqLen),dtype=np.int32)*PAD
    for i,seq in enumerate(input):
        for j,word in enumerate(seq):
            batchedInput[i][j] = word
    return batchedInput,inputSeqLen
        
def generateBatch(source,target,batch_size,epochs):
    numBatches = int(len(source)/batch_size)
    DecoderInputs = [[GO] + word for word in target ]
    DecoderTargets = [word + [EOS] for word in target]
    for it in range(epochs):
        for i in range(numBatches):
            encoderInputs,encoderInputLength = modifyBatch(source[i*batch_size:(i+1)*batch_size])
            decoderInputs,decoderInputLength = modifyBatch(DecoderInputs[i*batch_size:(i+1)*batch_size])
            decoderTargets,decoderTargetLength = modifyBatch(DecoderTargets[i*batch_size:(i+1)*batch_size])
            yield(encoderInputs,encoderInputLength,decoderInputs,decoderInputLength,decoderTargets,decoderTargetLength,i)

In [ ]:
timestamp = str(math.trunc(time.time()))
if not os.path.exists("seq_seq_checkpoints1"):
    os.mkdir("seq_seq_checkpoints1")
saver = tf.train.Saver(max_to_keep=1000)

In [ ]:
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)

In [ ]:
for encoderInputs,encoderInputLength,decoderInputs,decoderInputLength,decoderTargets,decoderTargetLength,batch_num in generateBatch(train,target,BATCH_SIZE,epoch):
    feedDict = {ENCODER_INPUTS:encoderInputs,EncoderSeqLength:encoderInputLength,DecoderSeqLength:decoderInputLength,DECODER_INPUTS:decoderInputs,decoder_targets:decoderTargets,lr:0.001}
    _,l = sess.run([optimiser,loss],feed_dict=feedDict)
    predict_ = sess.run(decoder_prediction,feed_dict=feedDict)
    print("Loss for batch i ", batch_num,l)
    if batch_num // 50 == 0:
        saved_file = saver.save(sess, 'seq_seq_checkpoints1/seq_seq_train' + timestamp, global_step=batch_num)
        print("Saved file: " + saved_file)
saved_file = saver.save(sess, 'seq_seq_checkpoints1/seq_seq_train' + timestamp, global_step=batch_num)
print("Saved file: " + saved_file) 
        
        

In [ ]:
##INFERENCE
decoderStartTokens = tf.placeholder(tf.int32,shape=[None], name="start_token")
decoderEndToken = EOS
decoderHelper = tf.contrib.seq2seq.GreedyEmbeddingHelper(decoder_embeddings,decoderStartTokens,decoderEndToken)
inference_decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell,decoderHelper,state,decoder_output_layer)
final_inference_outputs,final_inference_state,final_inference_sequence_lengths= tf.contrib.seq2seq.dynamic_decode(inference_decoder)


In [ ]:
test_input = [[0,2,4,6,8,10],[1,3,5,7,9,11,13,15,17,2]]
test_input,input_length = modifyBatch(test_input)
decoder_start_token = np.array([GO,GO])
feed_Dict = {ENCODER_INPUTS: test_input,EncoderSeqLength:np.array(input_length),decoderStartTokens:decoder_start_token}

In [ ]:
test_input

In [ ]:
final_inference_outputs

In [ ]:
#new_saver = tf.train.import_meta_graph('seq_seq_checkpoints1/seq_seq_train1500951341-49.meta')
#new_saver.restore(sess, "seq_seq_checkpoints1/seq_seq_train1500951341-49")
predict_test,seqLength = sess.run([final_inference_outputs,final_inference_sequence_lengths],feed_dict = feed_Dict)

print(predict_test,"output ")
print("Seq length",seqLength)

In [ ]:
predict_test[1]

In [ ]:
predict_test[0].shape

In [ ]:
###Attention


In [ ]:
predict_test

In [ ]:
seqLength